In [2]:
import cv2
import numpy as np
import easyocr

# Initialize EasyOCR
reader = easyocr.Reader(['en'], gpu=False)

# Load Haar Cascade for license plate detection
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_russian_plate_number.xml')

def detect_plates(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 20), maxSize=(300, 100))
    return plates

def process_frame(frame):
    plates = detect_plates(frame)
    
    for (x, y, w, h) in plates:
        plate_roi = frame[y:y+h, x:x+w]
        # Apply OCR on the detected plate region
        result = reader.readtext(plate_roi, detail=0)
        text = " ".join(result)
        
        # Draw rectangle around the plate and add the OCR text
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    return frame

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = process_frame(frame)
        cv2.imshow("License Plate Detection", frame)
        
        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "TestVideo.mp4"  # Specify the path to your video file here
    process_video(video_path)


Using CPU. Note: This module is much faster with a GPU.
